In [ ]:
%matplotlib inline

import re
import os
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.errors import SettingWithCopyWarning
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler


### Correlation Analysis:

In [ ]:
from sklearn.manifold import TSNE

strategies = ['None', 'Specific', 'Generic']
temperatures = ['0.0', '0.7']
variations = ['0', '1', '2']
recursions = ['0', '1']

regex_patterns = {
    'Powderkg': r'Powderkg = (\d+)',
    'wc': r'wc = ([\d.]+)',
    'materials': r'materials = ([\d./]+)',
    'curing': r'curing = (.+?)(?:\n|$)'
}


def find_explained_variance_using_pca(correlation_matrix, feature, strategy, temperature):
    pca = PCA(n_components=1)
    data = pd.DataFrame(correlation_matrix)
    pca.fit(data)
    first_principal_component = pca.components_[0]

    explained_variance_ratio = pca.explained_variance_ratio_
    print(f"Explained Variance Ratio for {feature}, {strategy} and {temperature}:", explained_variance_ratio)
    print(f"First Principal Component (Overall Relationship) for {feature}, {strategy} and {temperature}:",
          first_principal_component)


def perform_correlation_analysis(feature, strategy, temperature, recursion):
    correlation_matrix = df.corr()
    correlation_matrix = correlation_matrix.apply(lambda x: round(x, 2))
    average_cc_per_run = correlation_matrix.mean()
    correlation_matrix['Average Correlation'] = average_cc_per_run
    print('AVERAGE CORRELATION COEFFICIENT PER RUN')
    print(average_cc_per_run)
    overall_average_cc = correlation_matrix.stack().mean()
    print('OVERALL CORRELATION COEFFICIENT')
    print(overall_average_cc)
    # NaNs occur when the std is 0, i.e. we have constant functions. We handle this cas by replacing NaN with 1
    correlation_matrix.fillna(1, inplace=True)
    correlation_matrix.to_csv(
        f'Results/Correlation_Analysis/correlation_matrix_{feature}_{strategy}_{temperature}_{recursion}.csv')
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
    # Customize the plot (add labels, title, etc. if needed)
    plt.title(f'Correlation Heatmap {strategy} {temperature}')
    plt.show()
    # plt.savefig(
    #     f'Results/Correlation_Analysis/images/correlation_matrix_{feature}_{strategy}_{variation}_{temperature}_{recursion}.png',
    #     bbox_inches='tight')
    return correlation_matrix


def plot_individual_experiments(feature, strategy, temperature):
    plt.figure(figsize=(8, 6))
    plt.plot(df.index, df[f'{feature} 1'], label=f'{feature} Experiment 1')
    plt.plot(df.index, df[f'{feature} 2'], label=f'{feature} Experiment 2')
    plt.plot(df.index, df[f'{feature} 3'], label=f'{feature} Experiment 3')
    plt.legend()
    plt.title(f'Lineplot for {strategy} and {temperature}')
    plt.xlabel(feature)
    plt.tight_layout()
    plt.show()


def find_matching_files(pattern_0, pattern_1,pattern_2):
    filenames = []

    dir_name = 'Results/ID'
    for filename in os.listdir(dir_name):
        match = re.match(pattern_0, filename) or re.match(pattern_1, filename) or re.match(pattern_2, filename)
        if match:
            filenames.append(os.path.join(dir_name, filename))
    return filenames


def extract_data_for_given_run_for_all_variations(pattern_0, pattern_1, pattern_2):
    all_filenames = find_matching_files(pattern_0, pattern_1, pattern_2)
    all_runs_df = pd.DataFrame()
    warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
    for i, filename in enumerate(all_filenames):
        results_sample_df = pd.read_csv(filename)
        for col, pattern in regex_patterns.items():
            all_runs_df[f'{col} {i}'] = results_sample_df['Formulation'].str.extract(pattern, flags=re.DOTALL,
                                                                                     expand=False)
            if col.startswith('materials'):
                all_runs_df[f'{col} {i}'] = all_runs_df[f'{col} {i}'].str[:3]
            if col.startswith('curing'):
                all_runs_df[f'{col} {i}'][all_runs_df[f'{col} {i}'].astype(str).str.startswith('Heat')] = 0
                all_runs_df[f'{col} {i}'][all_runs_df[f'{col} {i}'].astype(str).str.startswith('heat')] = 0
                all_runs_df[f'{col} {i}'][all_runs_df[f'{col} {i}'].astype(str).str.startswith('Ambient')] = 1
                all_runs_df[f'{col} {i}'][all_runs_df[f'{col} {i}'].astype(str).str.startswith('ambient')] = 1

    return all_runs_df, all_filenames


def pca_and_corr_on_all_features(df):
    correlation_matrix = df.corr()
    correlation_matrix = correlation_matrix.apply(lambda x: round(x, 8))
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
    # Customize the plot (add labels, title, etc. if needed)
    plt.title('Correlation Heatmap between Lineplots')
    plt.show()
    pca = PCA(n_components=1)
    data = pd.DataFrame(correlation_matrix)
    pca.fit(data)
    # Get the first principal component
    first_principal_component = pca.components_[0]
    explained_variance_ratio = pca.explained_variance_ratio_
    print("Explained Variance Ratio:", explained_variance_ratio)
    print("First Principal Component (Overall Relationship):", first_principal_component)

In [ ]:
from itertools import product

all_combinations = list(product(strategies, temperatures, recursions))

#### Correlation Analyses between different runs

In [ ]:
for combination in all_combinations:
    strategy = combination[0]
    temperature = combination[1]
    recursion = combination[2]

    print(f'####### Run with strategy {strategy}, temperature {temperature} and recursion {recursion} #########')

    pattern_0 = fr'gpt-3.5-turbo_{strategy} 0_prompt_experiment_(\d+)_temp_{temperature}_target_(\d+)_\%_Dev_Budget_(\d+)_recursive_{recursion}_(\d+)\.csv'
    pattern_1 = fr'gpt-3.5-turbo_{strategy} 1_prompt_experiment_(\d+)_temp_{temperature}_target_(\d+)_\%_Dev_Budget_(\d+)_recursive_{recursion}_(\d+)\.csv'
    pattern_2 = fr'gpt-3.5-turbo_{strategy} 2_prompt_experiment_(\d+)_temp_{temperature}_target_(\d+)_\%_Dev_Budget_(\d+)_recursive_{recursion}_(\d+)\.csv'

    all_results_df, matching_filenames = extract_data_for_given_run_for_all_variations(pattern_0, pattern_1, pattern_2)


    warnings.filterwarnings("default")

    features = ['Powderkg', 'wc', 'materials', 'curing']
    for feature in features:
        print(f'################ Feature {feature} ################')
        data = {}
        for i in range(len(matching_filenames)):
            data[f'{feature} {i}'] = list(all_results_df[f'{feature} {i}'].values)

        df = pd.DataFrame(data)
        correlation_matrix = perform_correlation_analysis(feature, strategy, temperature, recursion)
        # plot_individual_experiments(feature, strategy, temperature)
        find_explained_variance_using_pca(correlation_matrix, feature, strategy, temperature)

        df_transpose = df.T
        if feature == 'Powderkg':
            all_features_df = df_transpose.values
        else:
            all_features_df = np.hstack((all_features_df, df_transpose.values))

    df = pd.DataFrame(all_features_df).T
    df.head(30)

    scaler = MinMaxScaler()
    scaler.fit(df)

    scaled_df = pd.DataFrame(scaler.transform(df), columns=df.columns)

    plt.plot(scaled_df.index, scaled_df[1].astype(str))
    plt.plot(scaled_df.index, scaled_df[2].astype(str))
    plt.gca().invert_yaxis()
    plt.show()